In [43]:
import os
import json
import time
import shelve
import requests
from dotenv import load_dotenv

from SpecialPrint import printSp, Style, Color

In [44]:
load_dotenv()

True

In [45]:
def fetchData():
    response = requests.get(os.getenv("USAGE_URL"))
    if response.status_code == 200:
        return response.json()
    return None

In [46]:
def readJsonData():
    with open("usage.json", "r") as file:
        data = json.load(file)
        return data

In [47]:
def updateJsonData():
    apiData = fetchData()
    oldData = readJsonData()

    oldDataIds = set(item['created_at'] for item in oldData)
    newData = [item for item in apiData if item['created_at'] not in oldDataIds]
    
    if newData or len(apiData) != len(oldData):
        updatedData = oldData + newData
        with open('usage.json', 'w') as file:
            json.dump(updatedData, file)

        return updatedData
    else:
        return oldData

In [48]:
def getTotals():
    data = readJsonData()

    calls = len(data)
    tokens = sum(
        (item["input_tokens"] if item["input_tokens"] is not None else 0)
        + (item["output_tokens"] if item["output_tokens"] is not None else 0)
        for item in data
    )
    spent = sum(item["price_of_request_in_cents"] for item in data) / 100

    totalsData = {
        "calls": calls,
        "tokens": tokens,
        "spent": spent
    }
    
    return totalsData

In [49]:
def updateShelfData():
    totalsData = getTotals()
    
    shelfData = {
        "calls": {"curr": 0, "diff": 0},
        "tokens": {"curr": 0, "diff": 0},
        "spent": {"curr": 0, "diff": 0},
    }

    with shelve.open("usage") as shelf:
        for key in ["calls", "tokens", "spent"]:
            if key not in shelf:
                shelf[key] = {"curr": 0, "diff": 0}
            else:
                currVal = shelf[key]["curr"]
                diffVal = totalsData[key] - currVal
                shelf[key] = {"curr": totalsData[key], "diff": diffVal}

            shelfData[key]["curr"] = shelf[key]["curr"]
            shelfData[key]["diff"] = shelf[key]["diff"]
    
    return shelfData

In [79]:
def readTotals():
    totals = {
        "totCalls": 0,
        "totTokens": 0,
        "totSpent": 0.0,
        "runCalls": 0,
        "runTokens": 0,
        "runSpent": 0.0,
    }
    with shelve.open("usage") as shelf:
        for key in ["calls", "tokens", "spent"]:
            totals[f"tot{key.capitalize()}"] = shelf[key]["curr"]
            totals[f"run{key.capitalize()}"] = shelf[key]["diff"]
    
    return totals

In [80]:
def printAndClear(message, sleepTime=1, newLine=False):
    if not newLine:
        print(f"\r{message}", end="", flush=True)
        time.sleep(sleepTime)
    else:
        print(f"\r{message}\n\n", end="")

In [109]:
def borderPrint(text1, text2=None):
    # padding = 10
    # width = (20 + len(text1)) if text2 is None else (20 + max(len(text1), len(text2)))
    # topBorder = printSp(f"{'= ' * int(width / 4)}\n", color=Color.BRIGHT_WHITE)
    # bottomBorder = printSp(f"\n{'= ' * int(width / 4)}", color=Color.BRIGHT_WHITE)
    
    # print(topBorder)
    # print(f"{text1.center(width)}")
    # if text2 is not None:
    #     print(f"{text2.center(width)}")
    # print(bottomBorder)
    
    padding = 10
    width = max(len(text1), len(text2) if text2 is not None else 0) + padding * 2
    
    topBorder = printSp(f"{'= ' * int(width // 4)}\n", color=Color.BRIGHT_WHITE)
    botBorder = printSp(f"\n{'= ' * int(width // 4)}", color=Color.BRIGHT_WHITE)
    
    print(topBorder)
    print(f"{text1.center(width)}")
    if text2 is not None:
        print(f"{text2.center(width)}")
    print(botBorder)

In [82]:
def showTotalUsage():
    tots = readTotals()
    
    calls = tots["totCalls"]
    tokens = tots["totTokens"]
    spent = tots["totSpent"]
    
    callsTxt = printSp("API CALLS:  ", color=Color.BRIGHT_WHITE)
    callsVal = printSp(f"{calls}", style=Style.BOLD, color=Color.BRIGHT_RED)
    tokensTxt = printSp("TOKENS:  ", color=Color.BRIGHT_WHITE)
    tokensVal = printSp(f"{tokens}", style=Style.BOLD, color=Color.BRIGHT_RED)
    spentTxt = printSp("SPENT:  ", color=Color.BRIGHT_WHITE)
    spentVal = printSp(f"{spent:.2f}", style=Style.BOLD, color=Color.BRIGHT_RED)

    divider = printSp("   ||   ", style=Style.BOLD, color=Color.BRIGHT_WHITE)

    printStr = f"{callsTxt}{callsVal}{divider}{tokensTxt}{tokensVal}{divider}{spentTxt}{spentVal}"

    return printStr

In [86]:
def showRunUsage():
    tots = readTotals()
    
    calls = tots["runCalls"]
    tokens = tots["runTokens"]
    spent = tots["runSpent"]
    
    callsVal = printSp(f"{calls}", style=Style.BOLD, color=Color.BRIGHT_BLUE)
    callsTxt = printSp(f" {"call" if calls == 1 else "calls"}", color=Color.BRIGHT_YELLOW)
    tokensVal = printSp(f"{tokens}", style=Style.BOLD, color=Color.BRIGHT_BLUE)
    tokensTxt = printSp(" tokens", color=Color.BRIGHT_YELLOW)
    spentVal = printSp(f"{spent:.2f}", style=Style.BOLD, color=Color.BRIGHT_BLUE)
    spentTxt = printSp(" spent", color=Color.BRIGHT_YELLOW)

    front = printSp("<<  ", color=Color.BRIGHT_YELLOW)
    back = printSp("  >>", color=Color.BRIGHT_YELLOW)
    divider = printSp("  ••  ", style=Style.BOLD, color=Color.BRIGHT_YELLOW)

    printStr = f"{front}{callsVal}{callsTxt}{divider}{tokensVal}{tokensTxt}{divider}{spentVal}{spentTxt}{back}"

    return printStr

In [89]:
def showUsage(withRun=False):

    totsText = showTotalUsage()
    runText = showRunUsage()
    
    if withRun:
        borderPrint(text1=totsText, text2=runText)
    else:
        borderPrint(text1=totsText)

In [110]:
showUsage(withRun=True)

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

            API CALLS:  88   ||   TOKENS:  288048   ||   SPENT:  502.90            
          <<  0 calls  ••  0 tokens  ••  0.00 spent  >>          

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 


In [55]:
def refreshUsageData():
    startTime = time.time()
    tryCount = 1
    checkCount = 4
    localData = readJsonData() 
    currTotals = findTotals()
    
    while time.time() - startTime < 20:
        newData = updateJsonData()
        totals = updateShelfData()
        if newData is not None:
            if len(newData) > len(localData):
                runCalls = totals["calls"]["diff"]
                runTokens = totals["tokens"]["diff"]
                runSpent = totals["spent"]["diff"]
                totCalls = totals["calls"]["curr"]
                totTokens = totals["tokens"]["curr"]
                totSpent = totals["spent"]["curr"]
                # printAndClear(f"Usage data successfully updated: {runCalls} {"call" if runCalls == 1 else "calls"}, {runTokens} tokens, {runSpent:.2f} spent", newLine=True)
                # showTotalUsage(totCalls, totTokens, totSpent)
                printAndClear(f"Usage data successfully updated with {runCalls} {"call" if runCalls == 1 else "calls"}", newLine=True)
                showUsage(totCalls, totTokens, totSpent, runCalls, runTokens, runSpent)
                return
            else:
                printAndClear(
                    f"Checking for new data || Attempt {tryCount} of {checkCount} .        "
                )
                printAndClear(
                    f"Checking for new data || Attempt {tryCount} of {checkCount} . .      "
                )
                printAndClear(
                    f"Checking for new data || Attempt {tryCount} of {checkCount} . . .    "
                )
                printAndClear(
                    f"Checking for new data || Attempt {tryCount} of {checkCount} . . . .  "
                )
                printAndClear(
                    f"Checking for new data || Attempt {tryCount} of {checkCount} . . . . ."
                )
                tryCount += 1
        else:
            printAndClear(
                f"Failed to connect with usage API. Retrying...", 5,
            )
            checkCount -= 1
    
    printAndClear(f"No new data detected.                              ", newLine=True)
    # showTotalUsage(currTotals["calls"], currTotals["tokens"], currTotals["spent"])
    showUsage(currTotals["calls"], currTotals["tokens"], currTotals["spent"], None, None, None)